In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np
import pandas as pd

In [ ]:
""" Carregando imagens do dataset """

data_path = "fruit-images"

IMG_SIZE = (100, 100)
BATCH_SIZE = 32

train_dir = "fruit_images/train"
test_dir = "fruit_images/test"

train_data = keras.preprocessing.image_dataset_from_directory(
    train_dir,
    seed=123,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

test_data = keras.preprocessing.image_dataset_from_directory(
    test_dir,
    seed=123,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

class_names = train_data.class_names

Found 1461 files belonging to 3 classes.
Found 490 files belonging to 3 classes.


In [ ]:
""" Criação e treinamento do modelo """

model = Sequential([
	Conv2D(32, (3, 3), activation='relu', input_shape=(100, 100, 3)),
	MaxPooling2D((2,2)),
	Conv2D(64, (3, 3), activation='relu'),
	MaxPooling2D((2,2)),
	Conv2D(64, (3, 3), activation='relu'),
	Flatten(),
	Dense(64, activation='relu'),
	Dense(10, activation='softmax')
])
model.summary()

c:\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 98, 98, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 49, 49, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 47, 47, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 23, 23, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 21, 21, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 28224)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │     1,806,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,863,370 (7.11 MB)

 Trainable params: 1,863,370 (7.11 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
history = model.fit(train_data, validation_data=test_data, epochs=15, callbacks=[early_stopping])

Epoch 1/15
46/46 ━━━━━━━━━━━━━━━━━━━━ 27s 470ms/step - accuracy: 0.6990 - loss: 47.5012 - val_accuracy: 1.0000 - val_loss: 1.1831e-04
Epoch 2/15
46/46 ━━━━━━━━━━━━━━━━━━━━ 39s 421ms/step - accuracy: 1.0000 - loss: 2.3064e-05 - val_accuracy: 1.0000 - val_loss: 2.8590e-04
Epoch 3/15
46/46 ━━━━━━━━━━━━━━━━━━━━ 20s 400ms/step - accuracy: 1.0000 - loss: 3.2734e-10 - val_accuracy: 1.0000 - val_loss: 1.7877e-04
Epoch 4/15
46/46 ━━━━━━━━━━━━━━━━━━━━ 25s 498ms/step - accuracy: 1.0000 - loss: 7.5771e-10 - val_accuracy: 1.0000 - val_loss: 1.7701e-04
Epoch 5/15
46/46 ━━━━━━━━━━━━━━━━━━━━ 31s 673ms/step - accuracy: 1.0000 - loss: 8.8610e-10 - val_accuracy: 1.0000 - val_loss: 1.7622e-04
Epoch 6/15
46/46 ━━━━━━━━━━━━━━━━━━━━ 41s 664ms/step - accuracy: 1.0000 - loss: 8.7500e-10 - val_accuracy: 1.0000 - val_loss: 1.7538e-04


In [ ]:
# Teste do modelo com imagens do dataset
from tensorflow.keras.preprocessing import image

def predict_image(img_path):
    img = image.load_img(img_path, target_size=(100, 100)) # Carregando imagem
    img_array = image.img_to_array(img) / 255.0 # Normalizando pixels
    img_array = np.expand_dims(img_array, axis=0) # Adicionando batch
    
    prediction = model.predict(img_array, verbose=0) # Previsão de classe da imagem
    predicted_class = class_names[np.argmax(prediction)] # Rótulo da classe
    
    print(f"- {predicted_class}")

test_image = "fruit_images/test/Orange/4_100.jpg"
print("Classe esperada: Orange")
predict_image(test_image)
test_image = "fruit_images/test/Apple/46_100.jpg"
print("Classe esperada: Apple")
predict_image(test_image)
test_image = "fruit_images/test/Banana/31_100.jpg"
print("Classe esperada: Banana")
predict_image(test_image)

Classe esperada: Orange
- Orange
Classe esperada: Apple
- Apple
Classe esperada: Banana
- Banana


In [ ]:
""" Aplicando métricas """

from sklearn import metrics

# Previsão das imagens do conjunto de teste
y_true, y_pred = [], []
for images, labels in test_data: 
    predictions = model.predict(images, verbose=0)
    y_pred.extend(np.argmax(predictions, axis=1))
    y_true.extend(labels.numpy())

y_true, y_pred = np.array(y_true), np.array(y_pred)

# Aplicando matriz de confusão
confusion_matrix = metrics.confusion_matrix(y_true, y_pred)

print(confusion_matrix)

# Aplicando demais métricas
print(metrics.classification_report(y_true, y_pred))

[[164   0   0]
 [  0 166   0]
 [  0   0 160]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       164
           1       1.00      1.00      1.00       166
           2       1.00      1.00      1.00       160

    accuracy                           1.00       490
   macro avg       1.00      1.00      1.00       490
weighted avg       1.00      1.00      1.00       490

